In [1]:
import azureml.core
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Experiment, Workspace
import azureml.core
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Experiment, Workspace

# Check core SDK version number
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")
print("")

# Log In to Azure ML Workspace
interactive_auth = InteractiveLoginAuthentication(tenant_id="76f90eb1-fb9a-4446-9875-4d323d6455ad")

ws = Workspace.from_config(auth=interactive_auth)
print('Workspace name: ' + ws.name, sep='\n')
from azureml.core import Dataset

# get the dataset from Azure ML Workspace
aml_dataset = Dataset.get_by_name(ws, 'dataset_final')
dataset = aml_dataset.to_pandas_dataframe()

You are currently using version 1.5.0 of the Azure ML SDK

Workspace name: team15


In [2]:
dataset.head()

,Column1,response_viber,stdev_days_between_visits_15d_div_response_sms,cheque_count_12m_g48_div_response_sms,k_var_cheque_3m_mul_k_var_days_between_visits_1m,cheque_count_12m_g48_div_response_viber,cheque_count_6m_g38_mul_k_var_cheque_3m,k_var_days_between_visits_1m_div_response_sms,food_share_1m_mul_k_var_days_between_visits_1m,k_var_count_per_cheque_6m_g27_div_response_sms,group,response_att,CardHolder
0,0,0.071429,1.850117,17.333334,1.214229,223.999989,7.4964,0.701892,0.210827,0.524875,1,0,16095858
1,1,0.000000,0.000000,1.000000,0.000000,NaN,0.9630,0.000000,0.000000,NaN,1,0,15906620
2,2,0.250000,0.000000,14.000000,NaN,56.000000,0.3295,NaN,NaN,0.576200,0,0,16495466
3,3,0.000000,0.000000,6.600000,0.000000,NaN,7.4665,0.000000,0.000000,0.362450,1,0,16570217
4,4,0.100000,1.414200,5.000000,0.424920,50.000000,0.9014,0.471400,0.135857,0.752600,1,0,16346871


In [3]:
import lightgbm
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm_notebook
import gc

In [4]:
def custom_metric(answers, take_top_ratio=0.25):
    answers.sort_values(by='uplift', inplace=True, ascending=False)
    n_samples = int(np.ceil(answers.shape[0] * take_top_ratio))
    answers = answers.iloc[:n_samples, :]
    answers_test = answers[answers['group'] == 'test']['response_att'].sum() / \
                   answers[answers['group'] == 'test'].shape[0]
    answers_control = answers[answers['group'] == 'control']['response_att'].sum() / \
                      answers[answers['group'] == 'control'].shape[0]
    return (answers_test - answers_control) * 100


In [5]:
dataset = dataset.drop('Column1', axis=1)

In [6]:
y = dataset.group == dataset.response_att
train = dataset.drop(['group', 'response_att'], axis=1)
train = train.set_index('CardHolder')

In [7]:
def score_ds(dataset_now, model, group, response_att):
    score = 0
    for t in range(6):
        k_fold = KFold(n_splits=2, shuffle=True)
        for train_index, test_index in k_fold.split(train.values):
            X_train, y_train = dataset_now.iloc[train_index], y.iloc[train_index]
            X_test, y_test = dataset_now.iloc[test_index], y.iloc[test_index]
            treatment_test = group.iloc[test_index].map({1: 'test', 0: 'control'})
            attr_test = response_att.iloc[test_index]
            model.fit(X_train, y_train)
            ans = pd.DataFrame({
                'uplift': model.predict_proba(X_test)[:, 1],
                'response_att': attr_test,
                'group': treatment_test
            })
            score += custom_metric(ans)
    return score/12

In [22]:
experiment = Experiment(workspace=ws, name="hyperparameters_tuning_2")
run =  experiment.start_logging(snapshot_directory=None)
run.log('model', 'target_changing')

In [23]:
params = []
scores = []
for boosting_type in ['gbdt', 'dart', 'goss']:
    for max_depth in [1, 2, 3]:
        for n_estimators in range(5, 131, 5):
            for class_weight in ['balanced', None]:
                model = LGBMClassifier(boosting_type=boosting_type, max_depth=max_depth, 
                                       n_estimators=n_estimators, class_weight=class_weight, n_jobs=-1)
                score = score_ds(train, model, dataset.group, dataset.response_att)
                run.log('boosting_type', boosting_type)
                run.log('max_depth', max_depth)
                run.log('n_estimators', n_estimators)
                run.log('class_weight', class_weight)
                run.log('score', score)
                print(boosting_type, max_depth, n_estimators, class_weight, score)
                params.append((boosting_type, max_depth, n_estimators, class_weight))
                scores.append(score)
run.complete()

gbdt 1 5 balanced 2.2720084569044583
gbdt 1 5 None 2.314742020367711
gbdt 1 10 balanced 2.543835077406426
gbdt 1 10 None 2.5747108668330387
gbdt 1 15 balanced 5.0879770862997775
gbdt 1 15 None 4.937704302456053
gbdt 1 20 balanced 5.45689740711119
gbdt 1 20 None 5.4313970391189335
gbdt 1 25 balanced 5.53716525675161
gbdt 1 25 None 5.574850877250782
gbdt 1 30 balanced 5.696926265210507
gbdt 1 30 None 5.766199223595456
gbdt 1 35 balanced 5.809054828324818
gbdt 1 35 None 5.800284919467649
gbdt 1 40 balanced 5.834948961258754
gbdt 1 40 None 5.824465909152984
gbdt 1 45 balanced 5.867567583417642
gbdt 1 45 None 5.896115886739618
gbdt 1 50 balanced 5.881391387718402
gbdt 1 50 None 5.882136888667893
gbdt 1 55 balanced 5.924538144308063
gbdt 1 55 None 5.8859166463884
gbdt 1 60 balanced 5.910628396777217
gbdt 1 60 None 5.914750877279821
gbdt 1 65 balanced 5.95204132822638
gbdt 1 65 None 5.940277077659574
gbdt 1 70 balanced 5.922036769279774
gbdt 1 70 None 5.930238325408703
gbdt 1 75 balanced 5.92

dart 2 65 None 6.08717024077551
dart 2 70 balanced 6.1295205007984945
dart 2 70 None 6.093414006008153
dart 2 75 balanced 6.09400894867579
dart 2 75 None 6.100246948988699
dart 2 80 balanced 6.111203999250595
dart 2 80 None 6.123586136089217
dart 2 85 balanced 6.079507824398635
dart 2 85 None 6.084602769122323
dart 2 90 balanced 6.112251611722779
dart 2 90 None 6.107514002183845
dart 2 95 balanced 6.102513024496173
dart 2 95 None 6.10316286200185
dart 2 100 balanced 6.123712204439479
dart 2 100 None 6.116184302189236
dart 2 105 balanced 6.139186217013829
dart 2 105 None 6.094786850843192
dart 2 110 balanced 6.110894666616599
dart 2 110 None 6.082437593125436
dart 2 115 balanced 6.126678009821137
dart 2 115 None 6.105129173116267
dart 2 120 balanced 6.108388402376581
dart 2 120 None 6.0808386673942
dart 2 125 balanced 6.106027923238531
dart 2 125 None 6.10356984523073
dart 2 130 balanced 6.108667840535062
dart 2 130 None 6.092921689093237
dart 3 5 balanced 5.575462356853531
dart 3 5 Non

goss 3 130 balanced 5.977139835570822
goss 3 130 None 6.026313109147069


In [24]:
pd.DataFrame({
    'parameters': params,
    'scores': scores
}).sort_values(by='scores').tail(10)

,parameters,scores
246,"(dart, 2, 100, balanced)",6.123712
299,"(dart, 3, 100, None)",6.124736
287,"(dart, 3, 70, None)",6.124914
252,"(dart, 2, 115, balanced)",6.126678
283,"(dart, 3, 60, None)",6.128396
234,"(dart, 2, 70, balanced)",6.129521
380,"(goss, 2, 45, balanced)",6.130237
116,"(gbdt, 3, 35, balanced)",6.134043
248,"(dart, 2, 105, balanced)",6.139186
298,"(dart, 3, 100, balanced)",6.144021


In [92]:
experiment = Experiment(workspace=ws, name="hyperparameters_tuning_5")
run =  experiment.start_logging(snapshot_directory=None)
run.log('model', 'single_model')

In [97]:
params = []
scores = []
for boosting_type in ['gbdt', 'dart', 'goss']:
    for max_depth in [2, 3]:
        for n_estimators in range(5, 131, 5):
            for class_weight in ['balanced', None]:
                model = LGBMClassifier(boosting_type=boosting_type, max_depth=max_depth, 
                                       n_estimators=n_estimators, class_weight=class_weight, n_jobs=-1)
                score = score_ds_single_model(train, model, dataset.group, dataset.response_att)
                run.log('boosting_type', boosting_type)
                run.log('max_depth', max_depth)
                run.log('n_estimators', n_estimators)
                run.log('class_weight', class_weight)
                run.log('score', score)
                print(boosting_type, max_depth, n_estimators, class_weight, score)
                params.append((boosting_type, max_depth, n_estimators, class_weight))
                scores.append(score)
run.complete()

/anaconda/envs/azureml_py36_automl/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda/envs/azureml_py36_automl/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda/envs/azureml_py36_automl/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

gbdt 2 5 balanced 0.557057578257817
gbdt 2 5 None 0.5535955720491598
gbdt 2 10 balanced 0.5605718756575813
gbdt 2 10 None 0.5602870004801433
gbdt 2 15 balanced 0.5589776891030948
gbdt 2 15 None 0.5579455078924097
gbdt 2 20 balanced 0.5623228018081413
gbdt 2 20 None 0.5599656385430417
gbdt 2 25 balanced 0.5610457736841264
gbdt 2 25 None 0.5658117971058834
gbdt 2 30 balanced 0.5592506381677024
gbdt 2 30 None 0.5611831594805234
gbdt 2 35 balanced 0.5613583306334607
gbdt 2 35 None 0.557839314474645
gbdt 2 40 balanced 0.5618756714263006
gbdt 2 40 None 0.5661502651524357


KeyboardInterrupt: 

In [8]:
experiment = Experiment(workspace=ws, name="hyperparameters_tuning_6")
run =  experiment.start_logging(snapshot_directory=None)
run.log('model', 'class_transform')

In [9]:
params = []
scores = []
for learning_rate in [0.1, 0.15, 0.25, 0.3]:
    for reg_lambda  in [0, 1e-3, 1e-2, 1e-1, 0.5]:
        for n_estimators in range(80, 110):
            seed = np.random.choice(1000)
            model = LGBMClassifier(boosting_type='dart', max_depth=3, 
                   n_estimators=n_estimators, class_weight='balanced', random_state=seed, 
                                  learning_rate=learning_rate, reg_lambda=reg_lambda, n_jobs=-1
                                  )
            score = score_ds(train, model, dataset.group, dataset.response_att)
            run.log('learning_rate', learning_rate)
            run.log('reg_lambda', reg_lambda)
            run.log('n_estimators', n_estimators)
            run.log('score', score)
            print(learning_rate, reg_lambda, n_estimators, score)
            params.append((learning_rate, reg_lambda, n_estimators))
            scores.append(score)
run.complete()

0.1 0 80 6.106801450004746
0.1 0 81 6.104596771846815
0.1 0 82 6.095178720440358
0.1 0 83 6.124824501164286
0.1 0 84 6.093109252107879
0.1 0 85 6.120643262021453
0.1 0 86 6.109151370181729
0.1 0 87 6.106284892264345
0.1 0 88 6.106093492856736
0.1 0 89 6.110100376582882
0.1 0 90 6.107161313059282
0.1 0 91 6.090900447419313
0.1 0 92 6.126183841098249
0.1 0 93 6.132120272885094
0.1 0 94 6.114990260380733
0.1 0 95 6.102215416373622
0.1 0 96 6.130267080106904
0.1 0 97 6.113259649242713
0.1 0 98 6.10971470109244
0.1 0 99 6.071926987272871
0.1 0 100 6.109530091437354
0.1 0 101 6.122348235998899
0.1 0 102 6.111667259236519
0.1 0 103 6.133554667367495
0.1 0 104 6.109861323322367
0.1 0 105 6.115006676904546
0.1 0 106 6.117636078638132
0.1 0 107 6.1395725311647125
0.1 0 108 6.105772855780846
0.1 0 109 6.114506770685871
0.1 0.001 80 6.1227860604294335
0.1 0.001 81 6.129302195813526
0.1 0.001 82 6.088142058273771
0.1 0.001 83 6.094543845245899
0.1 0.001 84 6.12236297715462
0.1 0.001 85 6.1098826814

0.15 0.5 84 6.104860783854744
0.15 0.5 85 6.112313115554547
0.15 0.5 86 6.101888363738797
0.15 0.5 87 6.074283468356761
0.15 0.5 88 6.086272846509112
0.15 0.5 89 6.10901075902238
0.15 0.5 90 6.105850695824451
0.15 0.5 91 6.106644158393753
0.15 0.5 92 6.103118870625864
0.15 0.5 93 6.099528604572659
0.15 0.5 94 6.090624091489353
0.15 0.5 95 6.060147131085153
0.15 0.5 96 6.103741454786591
0.15 0.5 97 6.124950707939251
0.15 0.5 98 6.06665682175511
0.15 0.5 99 6.1218874262574055
0.15 0.5 100 6.1070225506719815
0.15 0.5 101 6.092840949141249
0.15 0.5 102 6.124326616359183
0.15 0.5 103 6.09310890360554
0.15 0.5 104 6.097826875312034
0.15 0.5 105 6.082003424632462
0.15 0.5 106 6.10307996867786
0.15 0.5 107 6.091786775275852
0.15 0.5 108 6.111863895772362
0.15 0.5 109 6.087359146184046
0.25 0 80 6.085464600721991
0.25 0 81 6.087526380616501
0.25 0 82 6.0890219090060285
0.25 0 83 6.058987501740876
0.25 0 84 6.042211290436922
0.25 0 85 6.053138237058945
0.25 0 86 6.0995975874312345
0.25 0 87 6.09

KeyboardInterrupt: 

In [10]:
pd.DataFrame({
    'parameters': params,
    'scores': scores
}).sort_values(by='scores').tail(10)

,parameters,scores
196,"(0.15, 0.001, 96)",6.135395
132,"(0.1, 0.5, 92)",6.136887
248,"(0.15, 0.1, 88)",6.137830
217,"(0.15, 0.01, 87)",6.137858
180,"(0.15, 0.001, 80)",6.139300
154,"(0.15, 0, 84)",6.139460
27,"(0.1, 0, 107)",6.139573
153,"(0.15, 0, 83)",6.144708
101,"(0.1, 0.1, 91)",6.146186
114,"(0.1, 0.1, 104)",6.158868
